# ETL Pipeline for Immigration, Airport, and Cities Data
### Data Engineering Capstone Project

#### Project Summary <a id ='summary'></a> 

The goal of this project is to create an ETL pipeline using publicly available immigration data and city/airport data to create a database that is optimized for queries on immigration events. 

The project follows the below steps:
* [Step 1: Scope the Project and Gather Data](#step1)
* [Step 2: Explore and Assess the Data](#step2)
* [Step 3: Define the Data Model](#step3)
* [Step 4: Run ETL to Model the Data](#step4)
* [Step 5: Complete Project Write Up](#step5)

The following tables will be created:
* FACT Table with Immigration events
* DIM Table with Airport attributes
* DIM Table with City attributes

In order to facilitate queries, answering questions such as:
* Is there a correlation between the volume of travel and the number of entry ports?
* Does the volume of travel depend of the demographics of a given city?
* Which visa type holders comprise the majority of travelers in a given airport / city?


In [15]:
#Importing libraries
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from pyspark.sql import SparkSession
from pyspark.sql.functions import date_add, col

### Step 1: Scope the Project and Gather Data <a id ='step1'></a> 

#### Scope 
The goal of this project is to build an ETL, pulling data from 3 different sources and creating fact and dimension tables optimized for business analysis. PySpark and Pandas are the main tools used to process these data.

#### Data Description
* **I94 Immigration Data**: This data comes from the US National Tourism and Trade Office. A data dictionary (I94_SAS_Labels_Descriptions.SAS) is included in the current folder. [This](https://travel.trade.gov/research/reports/i94/historical/2016.html) is where the data comes from. There's a sample file (immigration_data_sample.csv) included here as well.
* **U.S. City Demographic Data**: This data comes from OpenSoft. You can read more about it [here](https://public.opendatasoft.com/explore/dataset/us-cities-demographics/export/).
* **Airport Code Table**: This is a simple table of airport codes and corresponding cities. It comes from [here](https://datahub.io/core/airport-codes#data).

In [16]:
#launching a spark session
spark = SparkSession.builder.\
config("spark.jars.repositories", "https://repos.spark-packages.org/").\
config("spark.jars.packages", "saurfang:spark-sas7bdat:2.0.0-s_2.11").\
enableHiveSupport().getOrCreate()

Writing immigration data:
* spark = spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')
* spark.write.parquet("sas_data")

In [17]:
#Reading in the data
c = pd.read_csv("us-cities-demographics.csv", sep = ';')
a = pd.read_csv("airport-codes.csv", sep = ',')
immigration = spark.read.parquet("sas_data") 

#Getting the immigration sample
i = pd.read_csv("immigration_data_sample.csv", sep = ',')


---

In [18]:
#Reading in the data dictionary for Immigration as helpfully suggested by a Udacity mentor
with open('./I94_SAS_Labels_Descriptions.SAS') as f:
    f_content = f.read()
    f_content = f_content.replace('\t', '')

def code_mapper(file, idx):
    f_content2 = f_content[f_content.index(idx):]
    f_content2 = f_content2[:f_content2.index(';')].split('\n')
    f_content2 = [i.replace("'", "") for i in f_content2]
    dic = [i.split('=') for i in f_content2[1:]]
    dic = dict([i[0].strip(), i[1].strip()] for i in dic if len(i) == 2)
    return dic

i94cit_res = code_mapper(f_content, "i94cntyl")
i94port = code_mapper(f_content, "i94prtl")
i94mode = code_mapper(f_content, "i94model")
i94addr = code_mapper(f_content, "i94addrl")
i94visa = {'1':'Business',
'2': 'Pleasure',
'3' : 'Student'}

In [19]:
#Turning a list of ports into a dataframe with distinct columns
i94port_df = pd.DataFrame.from_dict(i94port, orient='index')
i94port_df.reset_index(level=0, inplace=True)
i94port_df.columns = ['Code','City_State']
i94port_df

,Code,City_State
0,ALC,"ALCAN, AK"
1,ANC,"ANCHORAGE, AK"
2,BAR,"BAKER AAF - BAKER ISLAND, AK"
3,DAC,"DALTONS CACHE, AK"
4,PIZ,"DEW STATION PT LAY DEW, AK"
5,DTH,"DUTCH HARBOR, AK"
6,EGL,"EAGLE, AK"
7,FRB,"FAIRBANKS, AK"
8,HOM,"HOMER, AK"
9,HYD,"HYDER, AK"


In [20]:
#Splitting City_State column in two separate columns
i94port_df[['City', 'State','Else']] = i94port_df['City_State'].astype("str").str.split(', ', expand=True)
i94port_df.drop(['City_State', 'Else'], axis=1, inplace=True)
i94port_df

,Code,City,State
0,ALC,ALCAN,AK
1,ANC,ANCHORAGE,AK
2,BAR,BAKER AAF - BAKER ISLAND,AK
3,DAC,DALTONS CACHE,AK
4,PIZ,DEW STATION PT LAY DEW,AK
5,DTH,DUTCH HARBOR,AK
6,EGL,EAGLE,AK
7,FRB,FAIRBANKS,AK
8,HOM,HOMER,AK
9,HYD,HYDER,AK


In [21]:
#Counting % of missing ports out of total. Turns out to be 9%
i94port_df.query('City.str.contains("No PORT")', 
                 engine='python').count() / i94port_df.shape[0]

Code     0.090909
City     0.090909
State    0.000000
dtype: float64

In [22]:
#Creating a list of missing ports
missing_ports_list = list(i94port_df.query('City.str.contains("No PORT")', engine='python').Code)

---

### Step 2: Explore and Assess the Data <a id ='step2'></a> 

What I will be looking out for:

* **Completeness** (do we have all the records that we need? any missing / NaaN?)
* **Validity** (records that don’t conform to a defined schema, e.g. negative height not possible but present or duplicate key identifier)
* **Accuracy** (adheres to define schema, but is incorrect; e.g. overestimated values or out of date information)
* **Consistency** (data valid and accurate, but fields are represented in an inconsistent manner, e.g. state as NY and New York)
* **Tidiness** (structure of tidy data: variable = column, observation = row, observational unit = table)

In [23]:
c.head(5) #cities

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Wichita,Kansas,34.6,192354.0,197601.0,389955,23978.0,40270.0,2.56,KS,American Indian and Alaska Native,8791
1,Allen,Pennsylvania,33.5,60626.0,59581.0,120207,5691.0,19652.0,2.67,PA,Black or African-American,22304
2,Danbury,Connecticut,37.3,43435.0,41227.0,84662,3752.0,25675.0,2.74,CT,Black or African-American,8454
3,Nashville,Tennessee,34.1,314231.0,340365.0,654596,27942.0,88193.0,2.39,TN,Hispanic or Latino,67526
4,Stamford,Connecticut,35.4,64941.0,63936.0,128877,2269.0,44003.0,2.70,CT,Asian,11013


In [24]:
a.head(5) #airports

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11.0,NaN,US,US-PA,Bensalem,00A,NaN,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435.0,NaN,US,US-KS,Leoti,00AA,NaN,00AA,"-101.473911, 38.704022"
2,00AK,small_airport,Lowell Field,450.0,NaN,US,US-AK,Anchor Point,00AK,NaN,00AK,"-151.695999146, 59.94919968"
3,00AL,small_airport,Epps Airpark,820.0,NaN,US,US-AL,Harvest,00AL,NaN,00AL,"-86.77030181884766, 34.86479949951172"
4,00AR,closed,Newport Hospital & Clinic Heliport,237.0,NaN,US,US-AR,Newport,NaN,NaN,NaN,"-91.254898, 35.6087"


In [25]:
i.head(5) #immigration sample

,Unnamed: 0,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,count,dtadfile,visapost,occup,entdepa,entdepd,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,2027561,4084316.0,2016.0,4.0,209.0,209.0,HHW,20566.0,1.0,HI,20573.0,61.0,2.0,1.0,20160422,NaN,NaN,G,O,NaN,M,1955.0,07202016,F,NaN,JL,5.658267e+10,00782,WT
1,2171295,4422636.0,2016.0,4.0,582.0,582.0,MCA,20567.0,1.0,TX,20568.0,26.0,2.0,1.0,20160423,MTR,NaN,G,R,NaN,M,1990.0,10222016,M,NaN,*GA,9.436200e+10,XBLNG,B2
2,589494,1195600.0,2016.0,4.0,148.0,112.0,OGG,20551.0,1.0,FL,20571.0,76.0,2.0,1.0,20160407,NaN,NaN,G,O,NaN,M,1940.0,07052016,M,NaN,LH,5.578047e+10,00464,WT
3,2631158,5291768.0,2016.0,4.0,297.0,297.0,LOS,20572.0,1.0,CA,20581.0,25.0,2.0,1.0,20160428,DOH,NaN,G,O,NaN,M,1991.0,10272016,M,NaN,QR,9.478970e+10,00739,B2
4,3032257,985523.0,2016.0,4.0,111.0,111.0,CHM,20550.0,3.0,NY,20553.0,19.0,2.0,1.0,20160406,NaN,NaN,Z,K,NaN,M,1997.0,07042016,F,NaN,NaN,4.232257e+10,LAND,WT


In [26]:
#Checking how many rows are matching between the two datasets if we join on local_code.
#Seems to be an almost 100% match
len(a[~a['local_code'].isin(i['i94port'])])/a.shape[0]

0.9988561053109396

In [27]:
immigration.head(1) #top row from spark df

[Row(cicid=5748517.0, i94yr=2016.0, i94mon=4.0, i94cit=245.0, i94res=438.0, i94port='LOS', arrdate=20574.0, i94mode=1.0, i94addr='CA', depdate=20582.0, i94bir=40.0, i94visa=1.0, count=1.0, dtadfile='20160430', visapost='SYD', occup=None, entdepa='G', entdepd='O', entdepu=None, matflag='M', biryear=1976.0, dtaddto='10292016', gender='F', insnum=None, airline='QF', admnum=94953870030.0, fltno='00011', visatype='B1')]

In [28]:
#Getting to understand the number of rows and columns
print(f'count of rows and columns for cities: {c.shape}')
print(f'count of rows and columns for airports: {a.shape}')
print(f'count of rows and columns for immigration: {immigration.count(), len(immigration.columns)}')

count of rows and columns for cities: (2891, 12)
count of rows and columns for airports: (55075, 12)
count of rows and columns for immigration: (3096313, 28)


In [29]:
#Understanding columns and data types
c.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2891 entries, 0 to 2890
Data columns (total 12 columns):
City                      2891 non-null object
State                     2891 non-null object
Median Age                2891 non-null float64
Male Population           2888 non-null float64
Female Population         2888 non-null float64
Total Population          2891 non-null int64
Number of Veterans        2878 non-null float64
Foreign-born              2878 non-null float64
Average Household Size    2875 non-null float64
State Code                2891 non-null object
Race                      2891 non-null object
Count                     2891 non-null int64
dtypes: float64(6), int64(2), object(4)
memory usage: 271.1+ KB


In [30]:
a.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55075 entries, 0 to 55074
Data columns (total 12 columns):
ident           55075 non-null object
type            55075 non-null object
name            55075 non-null object
elevation_ft    48069 non-null float64
continent       27356 non-null object
iso_country     54828 non-null object
iso_region      55075 non-null object
municipality    49399 non-null object
gps_code        41030 non-null object
iata_code       9189 non-null object
local_code      28686 non-null object
coordinates     55075 non-null object
dtypes: float64(1), object(11)
memory usage: 5.0+ MB


In [31]:
i.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 29 columns):
Unnamed: 0    1000 non-null int64
cicid         1000 non-null float64
i94yr         1000 non-null float64
i94mon        1000 non-null float64
i94cit        1000 non-null float64
i94res        1000 non-null float64
i94port       1000 non-null object
arrdate       1000 non-null float64
i94mode       1000 non-null float64
i94addr       941 non-null object
depdate       951 non-null float64
i94bir        1000 non-null float64
i94visa       1000 non-null float64
count         1000 non-null float64
dtadfile      1000 non-null int64
visapost      382 non-null object
occup         4 non-null object
entdepa       1000 non-null object
entdepd       954 non-null object
entdepu       0 non-null float64
matflag       954 non-null object
biryear       1000 non-null float64
dtaddto       1000 non-null object
gender        859 non-null object
insnum        35 non-null float64
airline       967 non

In [32]:
c.columns.to_series().groupby(c.dtypes).groups

{dtype('int64'): Index(['Total Population', 'Count'], dtype='object'),
 dtype('float64'): Index(['Median Age', 'Male Population', 'Female Population', 'Number of Veterans', 'Foreign-born', 'Average Household Size'], dtype='object'),
 dtype('O'): Index(['City', 'State', 'State Code', 'Race'], dtype='object')}

In [33]:
c.describe()

,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,Count
count,2891.000000,2.888000e+03,2.888000e+03,2.891000e+03,2878.000000,2.878000e+03,2875.000000,2.891000e+03
mean,35.494881,9.732843e+04,1.017696e+05,1.989668e+05,9367.832523,4.065360e+04,2.742543,4.896377e+04
std,4.401617,2.162999e+05,2.315646e+05,4.475559e+05,13211.219924,1.557491e+05,0.433291,1.443856e+05
min,22.900000,2.928100e+04,2.734800e+04,6.321500e+04,416.000000,8.610000e+02,2.000000,9.800000e+01
25%,32.800000,3.928900e+04,4.122700e+04,8.042900e+04,3739.000000,9.224000e+03,2.430000,3.435000e+03
50%,35.300000,5.234100e+04,5.380900e+04,1.067820e+05,5397.000000,1.882200e+04,2.650000,1.378000e+04
75%,38.000000,8.664175e+04,8.960400e+04,1.752320e+05,9368.000000,3.397175e+04,2.950000,5.444700e+04
max,70.500000,4.081698e+06,4.468707e+06,8.550405e+06,156961.000000,3.212500e+06,4.980000,3.835726e+06


#### Missing values in Cities

In [34]:
#Declaring a function for missing values

def find_missing_data(df):
    '''
    INPUT:
        df - (dataframe), dataframe to check for missing values in its columns
    OUTPUT:
        df_null: (dataframe), with count & percentage of missing values in input dataframe columns
    '''
    null_data = df.isnull().sum()[df.isnull().sum() > 0]
    
    data_dict = {'count': null_data.values, 
                 'pct': np.round(null_data.values *100/df.shape[0],2)}
    
    df_null = pd.DataFrame(data=data_dict, index=null_data.index)
    df_null.sort_values(by='count', ascending=False, inplace=True)
    return df_null



In [35]:
c.isnull().sum().sum() #count of all missing values in Cities

48

In [36]:
find_missing_data(c) #getting missing vals by column

,count,pct
Average Household Size,16,0.55
Number of Veterans,13,0.45
Foreign-born,13,0.45
Male Population,3,0.10
Female Population,3,0.10


In [37]:
def nans(df): 
    return df[df.isnull().any(axis=1)]

nans(c) #missing val detail

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
5,San Juan,Puerto Rico,41.4,155408.0,186829.0,342237,NaN,NaN,NaN,PR,Hispanic or Latino,335559
9,Caguas,Puerto Rico,40.4,34743.0,42265.0,77008,NaN,NaN,NaN,PR,Hispanic or Latino,76349
280,Bayamón,Puerto Rico,39.4,80128.0,90131.0,170259,NaN,NaN,NaN,PR,Hispanic or Latino,169155
740,The Villages,Florida,70.5,NaN,NaN,72590,15231.0,4034.0,NaN,FL,White,72211
806,San Juan,Puerto Rico,41.4,155408.0,186829.0,342237,NaN,NaN,NaN,PR,American Indian and Alaska Native,4031
908,The Villages,Florida,70.5,NaN,NaN,72590,15231.0,4034.0,NaN,FL,Black or African-American,331
1121,Guaynabo,Puerto Rico,42.2,33066.0,37426.0,70492,NaN,NaN,NaN,PR,American Indian and Alaska Native,589
1196,Mayagüez,Puerto Rico,38.1,30799.0,35782.0,66581,NaN,NaN,NaN,PR,Asian,235
1443,Mayagüez,Puerto Rico,38.1,30799.0,35782.0,66581,NaN,NaN,NaN,PR,Hispanic or Latino,65521
1530,Caguas,Puerto Rico,40.4,34743.0,42265.0,77008,NaN,NaN,NaN,PR,American Indian and Alaska Native,624


In [38]:
#missing values comprise less than 1% of Cities data, so it's safe to drop them
c2 = c.dropna(axis=0)
c2.isnull().sum().sum() 

0

#### Missing values in Airport

In [39]:
a.isnull().sum().sum() #count of all missing values in Airports

126968

In [40]:
find_missing_data(a)

,count,pct
iata_code,45886,83.32
continent,27719,50.33
local_code,26389,47.91
gps_code,14045,25.50
elevation_ft,7006,12.72
municipality,5676,10.31
iso_country,247,0.45


In [41]:
#Dropping columns that are missing over 25% of data
cols = a.columns[a.isnull().sum()/len(a) > .50]
a2 = a.drop(cols,axis=1)
a2.head(2)

,ident,type,name,elevation_ft,iso_country,iso_region,municipality,gps_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11.0,US,US-PA,Bensalem,00A,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435.0,US,US-KS,Leoti,00AA,00AA,"-101.473911, 38.704022"


In [42]:
#Dropping the remaining rows with null values
a2.dropna(axis=0, inplace=True)
a2.isnull().sum().sum() #count of all missing values in Airports after modifications

0

#### Missing values in Immigration

In [43]:
type(immigration)

pyspark.sql.dataframe.DataFrame

In [44]:
i2 = immigration[~immigration["i94port"].isin(missing_ports_list)] #filtering out missng ports derived from the SAS file

{col:i2.filter(i2[col].isNull()).count() / i2.count() for col in i2.columns} #checking % of null values in each col

{'cicid': 0.0,
 'i94yr': 0.0,
 'i94mon': 0.0,
 'i94cit': 0.0,
 'i94res': 0.0,
 'i94port': 0.0,
 'arrdate': 0.0,
 'i94mode': 7.729379857947582e-05,
 'i94addr': 0.04910743449916529,
 'depdate': 0.045907341875969,
 'i94bir': 0.0002590474169964859,
 'i94visa': 0.0,
 'count': 0.0,
 'dtadfile': 3.2340501497688625e-07,
 'visapost': 0.6080088664718906,
 'occup': 0.9973755683034625,
 'entdepa': 7.697039356449892e-05,
 'entdepd': 0.0446046664756421,
 'entdepu': 0.999873548639144,
 'matflag': 0.0446046664756421,
 'biryear': 0.0002590474169964859,
 'dtaddto': 0.00015426419214397473,
 'gender': 0.13397279128927997,
 'insnum': 0.9632262625570083,
 'airline': 0.027045391187472068,
 'admnum': 0.0,
 'fltno': 0.0063222446377831495,
 'visatype': 0.0}

In [45]:
i2 = i2.drop("insnum", "entdepu", "occup", "visapost") #dropping extra columns
i2 = i2.dropna(how='any') #dropping null values
{col:i2.filter(i2[col].isNull()).count() / i2.count() for col in i2.columns}

{'cicid': 0.0,
 'i94yr': 0.0,
 'i94mon': 0.0,
 'i94cit': 0.0,
 'i94res': 0.0,
 'i94port': 0.0,
 'arrdate': 0.0,
 'i94mode': 0.0,
 'i94addr': 0.0,
 'depdate': 0.0,
 'i94bir': 0.0,
 'i94visa': 0.0,
 'count': 0.0,
 'dtadfile': 0.0,
 'entdepa': 0.0,
 'entdepd': 0.0,
 'matflag': 0.0,
 'biryear': 0.0,
 'dtaddto': 0.0,
 'gender': 0.0,
 'airline': 0.0,
 'admnum': 0.0,
 'fltno': 0.0,
 'visatype': 0.0}

#### Immigration data: setting up for Staging

In [46]:
i2 = i2.filter(i2.i94addr != 'other')
i3 = i2.select(col("cicid").alias("id"), 
                                       col("arrdate").alias("arrival_date"),
                                       col("i94port").alias("port_id"),
                                       col("i94addr").alias("state_id"),
                                       col("i94bir").alias("age"),
                                       col("gender").alias("gender"),
                                       col("i94visa").alias("visa_type"),
                                       "count").drop_duplicates()

i3.head()

Row(id=3493234.0, arrival_date=20563.0, port_id='WAS', state_id='DC', age=0.0, gender='M', visa_type=2.0, count=1.0)

In [47]:
type(i3.arrival_date)

pyspark.sql.column.Column

In [48]:
#Converting arrival date to DateTime
i3 = i3.withColumn('new_arr_date', col("arrival_date").cast("timestamp"))
i3 = i3.drop('arrival_date')
i3.head()

Row(id=3493234.0, port_id='WAS', state_id='DC', age=0.0, gender='M', visa_type=2.0, count=1.0, new_arr_date=datetime.datetime(1970, 1, 1, 5, 42, 43))

In [49]:
#Printing Immigration schema
i3.printSchema()

root
 |-- id: double (nullable = true)
 |-- port_id: string (nullable = true)
 |-- state_id: string (nullable = true)
 |-- age: double (nullable = true)
 |-- gender: string (nullable = true)
 |-- visa_type: double (nullable = true)
 |-- count: double (nullable = true)
 |-- new_arr_date: timestamp (nullable = true)



### Step 3: Define the Data Model <a id ='step3'></a> 
#### 3.1 Conceptual Data Model

The i94 immigration table contains a number of events / arrivals and will thus serve as **our fact table**:

* id,
* **port_id**, FK to join airport by (local_code)
* state_id,
* age,
* gender,
* visa_type,
* count,
* new_arr_date


For our dimension tables, we will have the City and the Airport attributes:

**dim_city**:
                                       
* **city**, FK
* **state_code**, FK
* **count**, FK
* median_age,
* male_population,
* female_population,
* total population
* veterans_count, 
* foreign_born,
* average_household_size,
* state,
* race,
*Compository key for joining to FACT: [City, State, Count]*

**dim_airport**:
* ident,
* **local_code** FK
* type,
* name,
* elevation_ft,
* state,
* country,
* municipality,
* airports_view


#### 3.2 Mapping Out Data Pipelines
1. Extract the data
2. Load all the datasets from CSV and SAS data files
3. Transform by dropping null values, coverting field format, cplitting columns etc.
4. Set up Staging --> create fact and dimenstion tables --> insert data accordingly 
5. Write them to parquet


### Step 4: Run Pipelines to Model the Data <a id ='step4'></a> 
#### 4.1 Create the data model

In [50]:
#Creating a temporary view for the fact table
i3.createOrReplaceTempView("imm_view")

In [53]:
#Creating a fact table
fact_table = spark.sql('''
                        SELECT id
                        ,'port_id'
                        ,'state_code'
                        ,'age'
                        ,'gender'
                        ,'visa_type'
                        ,'count'
                        ,'new_arr_date'
                        FROM imm_view
                        ''')

#Writing fact table to parquet files partitioned by port_code
fact_table.write.mode("append").partitionBy("port_id").parquet("/results/fact.parquet")

In [54]:
#Creating a dim table for Cities
c2[c2[['City', 'State Code', 'Count']].duplicated()].sort_values(by='City') #primary key will be compository
c2_spark = spark.createDataFrame(c2)
c2_spark.createOrReplaceTempView("cities_view")

dim_city = spark.sql('''
                                SELECT  City, 
                                        State, 
                                        `Median Age` AS median_age, 
                                        `Male Population` AS male_population, 
                                        `Female Population` AS female_population, 
                                        `Total Population` AS total_population, 
                                        `Number of Veterans` AS veterans_count, 
                                        `Foreign-born` AS foreign_born, 
                                        `Average Household Size` AS average_household_size, 
                                        `State Code` AS state_code, 
                                        Race, 
                                        Count
                                FROM cities_view
''')
dim_city.write.mode("append").partitionBy('City', 'state_code', 'Count').parquet("/results/dim.parquet")

In [55]:
#Creating a dim table for Airports
a2_spark = spark.createDataFrame(a2)
a2_spark.createOrReplaceTempView("airports_view")
dim_airport = spark.sql('''
                                SELECT 'ident', 
                                'type',
                                'name', 
                                'local_code',
                                'elevation_ft', 
                                SUBSTR('iso_region', 4) AS state, 
                                SUBSTR('iso_country', 4) AS country,
                                UPPER('municipality') AS municipality
                                FROM airports_view
                                WHERE 'municipality' IS NOT NULL
                                AND LOWER(TRIM('type')) NOT IN ('closed', 'heliport', 'seaplane_base', 'balloonport')
                                AND LENGTH('iso_region') = 5 ''')

dim_airport.write.parquet("/results/dim_a.parquet")

#### 4.2 Data Quality Checks

In [56]:
len(a2.index)

24387

In [57]:
def data_quality_check(df): 
    '''
    Input: dataframe
    Output: message confirming whether the dataframe has any rows
    '''
    result = df.shape[0]
    if result == 0:
        print("Zero records")
    else:
        print("Check passed")

# performing data quality checks
data_quality_check(c2)
data_quality_check(a2)

Check passed
Check passed


In [58]:
len(i3.columns) >0
i3.count()>0

True

#### 4.3 Data dictionary 

**Immigration: fact table**:

*  cicid      primary key, id from sas file
*  i94yr      entry year, 4 digit year
*  i94mon     entry month, numeric month
*  i94cit     i94 citizenship country code as per SAS Labels Descriptions file
*   i94res    i94 residence country code as per SAS Labels Descriptions file
*  i94port     i94port code as per SAS Labels Descriptions file
*  arrdate     date of arrival in U.S.
*  i94mode     code for travel mode of arrival as per SAS Labels Descriptions file
*  i94addr     address
*  depdate     departure date from U.S.
*  i94bir      age of the immigrant
*  i94visa     visa category code as per SAS Labels Descriptions file
*  dtadfile    Character Date Field - Date added to I-94 Files - CIC does not use */  
*  visapost    visa category code as per SAS Labels Descriptions file
*  occup       occupation of immigrant
*  entdepa     Arrival Flag - admitted or paroled into the U.S. - CIC does not use
*  entdepd     Departure Flag - Departed, lost I-94 or is deceased - CIC does not use
*  entdepu     Update Flag - Either apprehended, overstayed, adjusted to perm residence - CIC does not use
*  matflag     Match flag - Match of arrival and departure records
*  biryear     birth year of immigrant
* count        used for summary stats
*  dtaddto     character Date Field - Date to which admitted to U.S. (allowed to stay until) - CIC does not use */
*  gender      gender of immigrant
*  insnum      INS number
*  airline     airline code used to arrive in U.S.
*  admnum      admission number
*  fltno       flight number
*  visatype  visa type

**dim_city**:
* city  U.S. city name
* state U.S. city name
* median_age 
* male_population 
* female_population 
* total population
* veterans_count number of veterans
* foreign_born number of foreign born 
* average_household_size
* state_code
* race
* count

**dim_airport**:
* ident,
* type,
* name,
* elevation_ft,
* state,
* country,
* municipality,
* airports_view
 

#### Step 5: Complete Project Write Up <a id ='step5'></a> 
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.

As stated [at the beginning of the project](#summary), our main goal was to pull and combine disparate datasets that together would offer multifaceted responses to the business user who may be interested in learning more about the US travel and immigration/travel event correlation.

Since this data provide insights only once accumulated, I would recommend setting up an Airflow refresh no more frequent than once per quarter.

The reason I would recommend using Aiflow is for its simplicity of use, transparency that it brings to the ETL process, and free license. For data processing, I  stuck to Pandas and PySql as those are straightforward tools I'm very familiar with. They do the job, and do it well.





* Write a description of how you would approach the problem differently under the following scenarios:
    
 * **The data was increased by 100x. **
I would recommend moving to Amazon S3 for data hosting and Redshift for processing, creating maybe a 2-large node cluster. Judging from past experience, it's suited well to handle large datasets with ease.

 * **The data populates a dashboard that must be updated on a daily basis by 7am every day.**
 We would set our Airlfow to run daily as opposed to quarterly, with a failover message emailed to the ETL admin (me:) a few hours prior 7am.
 
 * **The database needed to be accessed by 100+ people.**
 I would warn against that as data can easily mislead if one's not careful (and you can't really trust that 100+ people would all arrive at the unbiased conclusions even if equipped with a data dictionary of sorts). It might make sense to construct a SSAS cube with predefined aggregations and publish the dataset to the cloud, along with some documentation explaining the backend calculations.